In [ ]:
import numpy as np
import pydicom
import os
import matplotlib.pyplot as plt
from glob import glob
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
from skimage import filters

import tensorflow as tf
from tensorflow.keras import Model, Sequential 
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Conv2D, MultiHeadAttention, AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D , Conv3D, Layer, MaxPooling2D, Dropout, Flatten, Dense, GRU, ConvLSTM2D, Input, BatchNormalization, TimeDistributed, MaxPooling3D, Bidirectional, LSTM
from tensorflow.keras.optimizers import Adam
from random import shuffle
from tensorflow.keras.regularizers import l2

from tensorflow.keras import layers
from tensorflow import keras
from keras.models import Model, load_model


import numpy as np
import os
import sys

import matplotlib.pyplot as plt
import pandas as pd
print(tf.__version__)
print(keras.__version__)
import pydicom
import re
import math
import random
# import bisect
np.random.seed(1234)
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
import warnings
warnings.filterwarnings(action='once')

MAX_SEQ_LENGTH = 200
NUM_FEATURES = 1024
IMG_SIZE = 512
BATCH_SIZE = 4
EPOCHS = 2
NUM_SCANS = 8
NUM_CHANNELS = 1
INPUT_DIM = 256

keras.utils.set_random_seed(1)
random.seed(1)
np.random.seed(1)

In [ ]:
fold = 0
all_ids = pd.read_csv('all_ids_updated.csv')
all_ids.ycoord = all_ids.ycoord.replace('True', '1.0').astype('float')
all_ids = all_ids.drop_duplicates('StudyInstanceUID')
fold_df = pd.read_csv('folds.csv')

directory = '/home/shared/nps/coat_np_0' + str(fold) + '/'

lisdir = os.listdir(directory)
print(len(lisdir))

files = pd.DataFrame({'file_name':lisdir})
files['StudyInstanceUID'] = files['file_name'].str.replace('.npy','')
files = pd.merge(files, all_ids)
files = files.drop(columns=['SeriesInstanceUID', 'SOPInstanceUID',
       'pe_present_on_image','ycoord', 'contains_lung'])
files = pd.merge(files, fold_df)
files = files[files.indeterminate == False].reset_index(drop=True)

In [ ]:
num_features = 64
MAX_SEQ_LENGTH = 200

class DataGenerator(Sequence):
    """Generates data for Keras
    Sequence based data generator. Suitable for building data generator for training and prediction.
    """
    def __init__(self, list_IDs, study_ids, num_features, directory,
                 seq_length, return_type = 'both', to_fit=True, batch_size=32, 
                 shuffle=True, full_set = None, random=False, set_type='test',
                 feat_type = 'feats', oversample=True, crop=False ):

        self.list_IDs = list_IDs
        self.study_ids = study_ids
        self.to_fit = to_fit
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_features = num_features
        self.seq_length = seq_length
        self.full_set = full_set
        self.return_type = return_type
        self.random = random
        warnings.filterwarnings(action='ignore')
        self.directory = directory
        self.set_type = set_type
        self.feat_type = feat_type
        self.oversample=oversample
        self.crop= crop
        
        if self.set_type == 'val':
            sample_pos = self.full_set[self.full_set.negative_exam_for_pe == False]
            sample_neg = self.full_set[self.full_set.negative_exam_for_pe == True]
            self.study_ids = pd.concat([sample_pos, sample_neg.sample(n=len(sample_pos))]).sample(frac = 1).reset_index(drop=True)
            self.list_IDs = np.arange(0, len(self.study_ids))
        self.on_epoch_end()
            
        
    def __len__(self):
        """Denotes the number of batches per epoch
        :return: number of batches per epoch
        """
        return int(np.floor(len(self.list_IDs) / self.batch_size))


    def __getitem__(self, index):
#         print('starting')
        indexes = self.list_IDs[index * self.batch_size:((index+1) * self.batch_size)]
        x_feats = np.zeros([self.batch_size, self.seq_length, self.num_features])
        y_seq = np.zeros([self.batch_size, self.seq_length, 1])
        y_tot = np.zeros([self.batch_size, 1])
        for i in range(0,self.batch_size):
            x, y = self._get_scan_data(self.study_ids.iloc[indexes[i]].StudyInstanceUID)
            x_feats[i] = x
            y_seq[i] = y[0]
            y_tot[i] = y[1]
            
        if self.to_fit:
            if self.return_type =='both':
                y = [np.array(y_seq), y_tot]
            elif self.return_type == 'seq':
                y = y_seq
            elif self.return_type == 'tot':
                y = y_tot
            else:
                print('valid return types are both, seq and tot')
                return False
            x = x_feats
            return x, y
        else:
            return (X)
        
    def on_epoch_end(self):
        """Updates indexes after each epoch
        """
        if self.set_type == 'train':
            sample_pos = self.full_set[self.full_set.negative_exam_for_pe == False]
            sample_neg = self.full_set[self.full_set.negative_exam_for_pe == True]
            if self.oversample == False:
                self.study_ids = pd.concat([sample_pos, sample_neg.sample(n=len(sample_pos))]).sample(frac=1).reset_index(drop=True)
            else:
                self.study_ids = pd.concat([sample_pos.sample(n=len(sample_neg), replace=True), sample_neg]).sample(frac = 1).reset_index(drop=True)
            self.list_IDs = np.arange(0, len(self.study_ids))
        if self.shuffle == True:
            np.random.shuffle(self.list_IDs)
            
    def _get_scan_data(self, study_id):
        scan = pd.DataFrame(np.load(self.directory +study_id + '.npy', allow_pickle=True).tolist())
        scan.ycoord = scan.ycoord.replace('True', '1.0').astype('float')
        scan = scan.sort_values(by=['ycoord'], ascending=False).reset_index(drop=True)
        
        features = scan.features.tolist()
        seq = scan.pe_present_on_image.tolist()
        seq = np.reshape(seq, [len(seq),1])
        tot = self.study_ids[self.study_ids.StudyInstanceUID == study_id].negative_exam_for_pe.iloc[0]
        
        if self.set_type == 'train' and self.random == True:
            new_len = int(len(scan) * (random.random()*0.25 +0.9))
            inst = np.round(np.arange(0,new_len)/new_len*len(features)).astype(int)
            features = (np.array(features)[inst]).tolist()
            seq = seq[inst]   
            
            new_len = int(len(features)*(1 - np.random.random() * 0.05))
            new_start = int(np.random.random() * (len(features)- new_len))
            features = features[new_start:new_start+new_len]
            seq = seq[new_start:new_start+new_len]        
        
        
        if len(features)>=self.seq_length:
            if self.crop==False:
                inst = np.round(np.arange(0,self.seq_length)/self.seq_length*len(features)).astype(int)
                xs = (np.array(features)[inst]).tolist()
                ys = seq[inst]
            if self.crop==True:
                xs = (np.array(features)[:self.seq_length]).tolist()
                ys = seq[:self.seq_length]
        else:
            xs = np.zeros([self.seq_length, self.num_features])
            ys = np.zeros([self.seq_length, 1])

            start = int(np.floor((self.seq_length - len(features))/2))
            end = start + len(features)
            xs[start:end] = features
            ys[start:end] = seq

        ys = ys.tolist()
                    
        return (xs, [ys, tot])

    
def get_generators(df, fold, batch_size, out, feat, random, seq_len=200, crop=True, np_dir='/home/shared/coat_np_0', osa=True, test_only=False):

    dire = np_dir + str(fold) + '/'

    if test_only:
        return DataGenerator(np.arange(0, len(df)),
                                    df, 
                                    64,
                                    dire,
                                    seq_len,
                                    batch_size=batch_size,
                                    set_type = 'test',
                                    return_type=out,
                                    feat_type=feat,
                                    crop=crop,
                                    shuffle=False)
    
    test_df = df[df.fold == fold]
    train_df = df[df.fold != fold]
        
    print(train_df.fold.unique())
    print(test_df.fold.unique())
    test_generator = DataGenerator(np.arange(0, len(test_df)),
                                    test_df, 
                                    64,
                                    dire,
                                    seq_len,
                                    batch_size=batch_size,
                                    set_type = 'test',
                                    return_type=out,
                                    feat_type=feat,
                                    crop=crop)
    
    val_generator = DataGenerator(np.arange(0, len(test_df)),
                                  test_df, 
                                  64,
                                  dire,
                                  seq_len,
                                  batch_size=batch_size,
                                  set_type ='val',
                                  full_set=test_df,
                                  return_type=out,
                                  feat_type=feat,
                                  crop=crop)
    
    train_generator = DataGenerator(np.arange(0, len(train_df)),
                                  train_df, 
                                  64,
                                  dire,
                                  seq_len,
                                  batch_size=batch_size,
                                  set_type ='train',
                                  full_set = train_df,
                                  random = random,
                                  return_type=out,
                                  feat_type=feat,
                                  crop=crop,
                                  oversample=osa)
    
    return train_generator, val_generator, test_generator

In [ ]:
def get_attn_lstm(seq_len, noise=0.001):
        num_heads = 1
        key_dim = 8
        inputs = keras.Input(shape=(seq_len, 64))
        x = layers.BatchNormalization()(inputs)
        x = layers.GaussianNoise(noise)(x)
        x = layers.TimeDistributed(layers.Dropout(0.3))(x)
        attn = layers.MultiHeadAttention(num_heads=num_heads,
                                                     key_dim=key_dim, 
                                                     dropout=0.4
                                                    )(x, x)
        attn = layers.LayerNormalization()(attn)
        x = layers.Add()((x,attn))
        x = layers.TimeDistributed(layers.Dropout(0.4))(x)
        x = layers.Bidirectional(layers.LSTM(units=8, 
                                             return_sequences=True,
                                             dropout=0.3, 
                                             recurrent_regularizer=keras.regularizers.L2(0.01),
                                            ))(x)
        x = layers.TimeDistributed(layers.Dropout(0.4))(x)
        x = layers.LSTM(units=8, return_sequences=True,
                        dropout=0.3, 
#                         bias_regularizer=keras.regularizers.L2(0.005)
                       )(x)
        x = layers.TimeDistributed(layers.Dropout(0.4))(x)
        x = layers.Bidirectional(layers.LSTM(units=8,
                                             return_sequences=False, 
                                             dropout=0.3,
#                                              bias_regularizer=keras.regularizers.L2(0.005),
                                            ))(x)
        x = layers.Dropout(0.4)(x)
        stack_outputs = layers.Dense(1, activation='sigmoid', name='tot_out')(x)

        model = keras.models.Model(inputs=inputs, outputs=stack_outputs)
        

        return model

In [ ]:
seq_len = 208
from sklearn.metrics import confusion_matrix
from sklearn import metrics

rs = 4 
folds = range(0,10)
for fold in folds:        
    tf.keras.backend.clear_session()

    keras.utils.set_random_seed(rs)
    random.seed(rs)
    np.random.seed(rs)

    model = get_attn_lstm(seq_len, noise=0.005)

    np_dir = '/home/shared/nps/coat_np_0'

    batch_size = 32
    training_gen, validation_gen, test_gen = get_generators(files, fold, 
                                                            batch_size, 
                                                            'tot', 
                                                            'feats', 
                                                            True, 
                                                            seq_len=seq_len, 
                                                            np_dir=np_dir, 
                                                            crop=False,
                                                            osa=False
                                                           )

    lr = 0.001
    opt = keras.optimizers.Nadam(learning_rate=lr)#,beta_1=0.9, beta_2=0.999, decay=0.01)
    scheduler = keras.optimizers.schedules.ExponentialDecay(lr, 2000,0.99)
    scheduler = keras.callbacks.LearningRateScheduler(scheduler)

    es = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

    model.compile(
            optimizer=opt,
            loss=keras.losses.binary_crossentropy,
            metrics=["accuracy",]
        )

    model.summary()

    checkpoint_filepath = '/home/shared/model_checkpoint_paige/scan/reg5-kfold_0'+str(fold)
    checkpoint = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    history = model.fit(
        training_gen, 
        validation_data = validation_gen,
        epochs=15,
        callbacks=[checkpoint,scheduler,
                   es
                  ],
    )

    df_hist = pd.DataFrame(history.history)
    df_hist.to_csv('/home/shared/model_checkpoint_paige/scan/hist-reg5-kfold_0'+str(fold)+'.csv')

    _,_, test_gen = get_generators(files, fold,  1,  'tot', 'feats', 
                                   True,  seq_len=seq_len, np_dir=np_dir, 
                                   crop=False, osa=False)

    t_len = test_gen.__len__()
    x_test = np.zeros([t_len,208,num_features])
    y_test = np.zeros([t_len,1])
    for i in range(0,t_len):
        x_test[i],y= test_gen.__getitem__(i)
        y_test[i] = y

    model.load_weights(checkpoint_filepath)
    y_pred = model(x_test, training=False)

    cm = confusion_matrix(y_test, np.asarray(y_pred).round())

    sepecifity = cm[1][1]/(cm[1][1]+cm[1][0])
    sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
    accuracy = (cm[1][1]+cm[0][0])/(cm[1][1]+cm[1][0]+cm[0][0]+cm[0][1])
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
    roc_auc = metrics.auc(fpr, tpr)

    print(cm)
    print(sensitivity, sepecifity, accuracy, roc_auc)    


In [ ]:
seq_len=208
dos5 = pd.DataFrame(columns=['fold','auc','sensitivity', 'specificity', 'accuracy','npv','ppv'])

plt.figure(0).clf()
for fold in range(0,10):

    model = get_attn_lstm(seq_len)

    np_dir = '/home/shared/nps/coat_np_0'

    batch_size = 1
    _,_, test_gen = get_generators(files, fold, 
                                   batch_size, 
                                   'tot', 
                                   'feats', 
                                   True, 
                                   seq_len=seq_len, 
                                   np_dir=np_dir, 
                                   crop=False,
                                   osa=False)
    
    
    t_len = test_gen.__len__()
    x_test = np.zeros([t_len,208,num_features])
    y_test = np.zeros([t_len,1])
    for i in range(0,t_len):
        x_test[i],y= test_gen.__getitem__(i)
        y_test[i] = y
        
    lr = 0.001
    opt = keras.optimizers.Nadam(learning_rate=lr)
    model.compile(
            optimizer=opt,
            loss=keras.losses.binary_crossentropy,
            metrics=["accuracy",]
        )

    checkpoint_filepath = '/home/shared/model_checkpoint_paige/scan/reg5-kfold_0'+str(fold)
    model.load_weights(checkpoint_filepath).expect_partial()
    
    (fpr,tpr), data, cm = test_version(model, x_test,y_test)
    
    auc = metrics.auc(fpr, tpr)
    plt.plot(fpr,tpr, label='Fold '+str(fold +1)+' AUC={:.4f}'.format(auc))
    
    dos5 = dos5.append({'fold':fold,'auc':data[0],'sensitivity':data[1], 'specificity':data[2], 'accuracy':data[3], 'npv':data[4],'ppv':data[5]}, ignore_index=True)
plt.legend()
plt.show()
print(dos5)